<a href="https://colab.research.google.com/github/RahmiaWati431/KematanganTomatMenggunakanMachineLearning/blob/main/UjiSVM70_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from IPython.display import display
import pandas as pd
from statsmodels.stats.contingency_tables import mcnemar

# --- Load Data ---
crop_df = pd.read_csv('/content/drive/MyDrive/Penelitian/Skripsi/Crop/70.30/Prediksi/svm/Prediksi_SVM_Crop_70:30.csv')
nocrop_df = pd.read_csv('/content/drive/MyDrive/Penelitian/Skripsi/TanpaCrop/70.30/Prediksi/svm/Prediksi_SVM_TanpaCrop_70:30.csv')

# --- Pastikan sejajar berdasarkan Filename ---
crop_df.sort_values(by='Filename', inplace=True)
nocrop_df.sort_values(by='Filename', inplace=True)
crop_df.reset_index(drop=True, inplace=True)
nocrop_df.reset_index(drop=True, inplace=True)

# --- Pastikan label asli sama ---
assert all(crop_df['Actual'] == nocrop_df['Actual']), "⚠️ Ground truth tidak cocok!"

# --- Buat tabel evaluasi lengkap ---
evaluasi_df = pd.DataFrame({
    'Label Asli': crop_df['Actual'],
    'SVM Crop Prediksi': crop_df['Predicted_SVM_Crop'],
    'SVM TanpaCrop Prediksi': nocrop_df['Predicted_SVM_Crop'],
})

# Tambahkan kolom kebenaran prediksi
evaluasi_df['Crop Benar?'] = evaluasi_df['Label Asli'] == evaluasi_df['SVM Crop Prediksi']
evaluasi_df['TanpaCrop Benar?'] = evaluasi_df['Label Asli'] == evaluasi_df['SVM TanpaCrop Prediksi']

# --- Hitung nilai a, b, c, d ---
a = sum((evaluasi_df['Crop Benar?']) & (evaluasi_df['TanpaCrop Benar?']))   # Sama-sama benar
b = sum((evaluasi_df['Crop Benar?']) & (~evaluasi_df['TanpaCrop Benar?']))  # Crop benar, TanpaCrop salah
c = sum((~evaluasi_df['Crop Benar?']) & (evaluasi_df['TanpaCrop Benar?']))  # Crop salah, TanpaCrop benar
d = sum((~evaluasi_df['Crop Benar?']) & (~evaluasi_df['TanpaCrop Benar?'])) # Sama-sama salah

# --- Cetak tabel kontingensi lengkap ---
table_full = [[a, b],
              [c, d]]

print("\n📋 Tabel Kontingensi McNemar (Lengkap a,b,c,d):")
print(pd.DataFrame(table_full,
                   index=['Crop Benar', 'Crop Salah'],
                   columns=['TanpaCrop Benar', 'TanpaCrop Salah']))

# --- Simpan tabel evaluasi ke Excel ---
output_excel = '/content/drive/MyDrive/Penelitian/Skripsi/CropvsNoCrop/70.30/svm/Evaluasi_McNemar_SVM_Crop_vs_TanpaCrop_70:30.xlsx'
evaluasi_df.to_excel(output_excel, index=False)
print(f"\n✅ Tabel evaluasi disimpan ke: {output_excel}")

# --- Uji McNemar hanya pakai b & c ---
table_bc = [[0, b],
            [c, 0]]

result = mcnemar(table_bc, exact=True)

print(f"\n📊 Hasil Uji McNemar:")
print(f"Statistik : {result.statistic}")
print(f"p-value   : {result.pvalue:.4f}")

# Interpretasi
alpha = 0.05
if result.pvalue < alpha:
    print("💡 Kesimpulan: Terdapat perbedaan signifikan antara model SVM Crop dan SVM TanpaCrop (p < 0.05)")
else:
    print("💡 Kesimpulan: Tidak terdapat perbedaan signifikan antara model SVM Crop dan SVM TanpaCrop (p >= 0.05)")



📋 Tabel Kontingensi McNemar (Lengkap a,b,c,d):
            TanpaCrop Benar  TanpaCrop Salah
Crop Benar               56                6
Crop Salah                1                0

✅ Tabel evaluasi disimpan ke: /content/drive/MyDrive/Penelitian/Skripsi/CropvsNoCrop/70.30/svm/Evaluasi_McNemar_SVM_Crop_vs_TanpaCrop_70:30.xlsx

📊 Hasil Uji McNemar:
Statistik : 1.0
p-value   : 0.1250
💡 Kesimpulan: Tidak terdapat perbedaan signifikan antara model SVM Crop dan SVM TanpaCrop (p >= 0.05)


# New Section